In [1]:
#Import required libraries
import gspread
import sys
#from oauth2client.service_account import ServiceAccountCredentials
from google.auth.transport.requests import AuthorizedSession
from google.oauth2 import service_account
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
from gspread_dataframe import set_with_dataframe
from gspread_dataframe import get_as_dataframe

In [2]:
#Import shared functions
sys.path.append('..')
from IPM_Shared_Code_public.Python.google_creds_functions import create_assertion_session
from IPM_Shared_Code_public.Python.utils import get_config
from IPM_Shared_Code_public.Python.delta_functions import *
from IPM_Shared_Code_public.Python.sql_functions import *

C:\ProgramData\Anaconda3\lib\site-packages\authlib\client\__init__.py:12: AuthlibDeprecationWarning: Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.
It will be compatible before version 1.0.
Read more <https://git.io/Jeclj#file-rn-md>
  deprecate('Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.', '1.0', 'Jeclj', 'rn')


### Use the config file to setup connections

In [3]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
dwh = config['db']['crowdsdb']
cred_file = config['google']['path_to_file']

In [4]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + dwh + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

### Send the data to the sheet

In [5]:
sql = 'select * from crowdsdb.dbo.vw_consolidated_socialdistancing'

In [6]:
consolidated_sql = (pd.read_sql(con = engine, sql = sql)
                    .fillna(value = np.nan, axis = 1))

In [7]:
consolidated_sql.tail()

,source_survey,encounter_datetime,gispropnum,reported_as,site_id,site_desc,park_borough,location_adddesc,city_agency,encounter_type,amenity,patroncount,police_precinct,police_boro_com,communityboard
40397,DPR Crowds,2020-06-13 10:54:57,Q051,Q051,Q051,St. Albans Park,Queens,NaN,DPR,NaN,Adult fitness equipment,15.0,113,Queens South,412
40398,DPR Crowds,2020-06-13 10:38:20,Q300A,Q300A,Q300A-01,Kissena Corridor Park-Underhill Playground,Queens,NaN,DPR,NaN,Playground,50.0,111,Queens North,411
40399,DPR Crowds,2020-06-13 10:32:48,M077,M077,M077-ZN02,St Nicholas Park Zone 02,Manhattan,NaN,DPR,NaN,Bench/sitting area,18.0,26,Manhattan North,109
40400,DPR Crowds,2020-06-13 10:07:32,Q015,Q015,Q015-ZN04,Forest Park-Victory Field,Queens,NaN,DPR,NaN,Soccer field,40.0,NaN,NaN,482
40401,DPR Crowds,2020-06-13 09:31:19,X010A,X010A,X010A,Tremont Park,Bronx,NaN,DPR,NaN,Bench/sitting area,12.0,48,Bronx,206


In [8]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
#creds = ServiceAccountCredentials.from_json_keyfile_name(cred_file, scope)
#client = gspread.authorize(creds)

In [ ]:
creds = service_account.Credentials.from_service_account_file(cred_file).with_scopes(scope)

In [ ]:
authed_session = AuthorizedSession(creds)

In [ ]:
client = gspread.Client(creds, authed_session)

In [9]:
sheet = client.open('consolidated_social_distancing_data')

In [10]:
ws = sheet.worksheet('Data')

In [11]:
set_with_dataframe(ws, consolidated_sql, include_index = False, 
                   include_column_header = True, resize = True, allow_formulas = False)